# Ekstraksi dan Pengolahan Data Kepemilikan Tanah

## 1. Menyiapkan coding environment, termasuk menginstall dan/atau memperbarui library

In [1]:
# Menginstall library-library
!pip install pandas
!pip install pytesseract
!pip install pdf2image
!pip install pillow
!pip install numpy
!pip install matplotlib

# Mengimport library-library untuk digunakan
import cv2
import numpy as np
import os
import pandas as pd
import pytesseract
from pdf2image import convert_from_path
from PIL import Image
from matplotlib import pyplot as plt

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Loading the preprocessed template CSV file
template = pd.read_csv('DATA C-DESA NOBOREJO preprocessed.csv')

#### !!! TESTING "berkas 500-520 EXAMPLE 2.pdf !!!

##### PDF to PNG conversion

In [3]:
# Initialize a new empty dataframe with the same columns as the template
df = pd.DataFrame(columns=template.columns)
df.head()

# Save CSV to the output directory
output_dir = 'Berkas Files/images of berkas 500-520 EXAMPLE 2/Preprocessed halaman 1 stuff'
output_file = 'output.csv'
df.to_csv(os.path.join(output_dir, output_file), index=False)

In [4]:
# Create subdirectory to hold images inside the Berkas Files subdirectory
output_folder = 'Berkas Files/images of berkas 500-520 EXAMPLE 2'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Convert PDF to images
pdf_path = 'Berkas Files/berkas 500-520 EXAMPLE 2.pdf'
images = convert_from_path(pdf_path)

# Save each image to the output folder
for i, image in enumerate(images):
    image_path = os.path.join(output_folder, f'halaman_{i+1}.png')
    image.save(image_path, 'PNG')

# Display the paths of the saved images
image_paths = [os.path.join(output_folder, f'halaman_{i+1}.png') for i in range(len(images))]
print("Images saved at:")
for path in image_paths:
    print(path)

Images saved at:
Berkas Files/images of berkas 500-520 EXAMPLE 2\halaman_1.png
Berkas Files/images of berkas 500-520 EXAMPLE 2\halaman_2.png
Berkas Files/images of berkas 500-520 EXAMPLE 2\halaman_3.png
Berkas Files/images of berkas 500-520 EXAMPLE 2\halaman_4.png
Berkas Files/images of berkas 500-520 EXAMPLE 2\halaman_5.png
Berkas Files/images of berkas 500-520 EXAMPLE 2\halaman_6.png
Berkas Files/images of berkas 500-520 EXAMPLE 2\halaman_7.png
Berkas Files/images of berkas 500-520 EXAMPLE 2\halaman_8.png
Berkas Files/images of berkas 500-520 EXAMPLE 2\halaman_9.png
Berkas Files/images of berkas 500-520 EXAMPLE 2\halaman_10.png
Berkas Files/images of berkas 500-520 EXAMPLE 2\halaman_11.png
Berkas Files/images of berkas 500-520 EXAMPLE 2\halaman_12.png
Berkas Files/images of berkas 500-520 EXAMPLE 2\halaman_13.png
Berkas Files/images of berkas 500-520 EXAMPLE 2\halaman_14.png
Berkas Files/images of berkas 500-520 EXAMPLE 2\halaman_15.png
Berkas Files/images of berkas 500-520 EXAMPLE 2

##### Image preprocessing

In [5]:
# Load the image
image_path = 'Berkas Files/images of berkas 500-520 EXAMPLE 2/halaman_1.png'
image = cv2.imread(image_path, cv2.IMREAD_COLOR)  # Use IMREAD_COLOR to load properly

if image is None:
    raise ValueError(f"Image not loaded. Check if the path is correct: {image_path}")

In [25]:
# Step 1: Apply refined adaptive thresholding
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale (just in case)
adaptive_binary = cv2.adaptiveThreshold(
    gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 101, 11
)

# Save the output for this step
step_1_path = os.path.join(output_dir, 'Step 1 - Refined Adaptive Thresholded Image.png')
cv2.imwrite(step_1_path, adaptive_binary)

print(f"Step 1 completed. Refined adaptive thresholded image saved at: {step_1_path}")


Step 1 completed. Refined adaptive thresholded image saved at: Berkas Files/images of berkas 500-520 EXAMPLE 2/Preprocessed halaman 1 stuff\Step 1 - Refined Adaptive Thresholded Image.png


In [22]:
# Step 2: Remove unwanted hand region
# Adjust the bounding box to cover the hand
hand_region = (0, adaptive_binary.shape[0] - 800, 600, 1000)  # x, y, width, height
x, y, w, h = hand_region

# Mask the hand by painting the region white
hand_removed = adaptive_binary.copy()
cv2.rectangle(hand_removed, (x, y), (x + w, y + h), (255, 255, 255), -1)

# Save the cleaned image
step_2_path = os.path.join(output_dir, 'Step 2 - Hand Removed.png')
cv2.imwrite(step_2_path, hand_removed)

print(f"Step 2 completed. Hand removed image saved at: {step_2_path}")

Step 2 completed. Hand removed image saved at: Berkas Files/images of berkas 500-520 EXAMPLE 2/Preprocessed halaman 1 stuff\Step 2 - Hand Removed.png


In [78]:
# Step 3: Highlight table lines in green (retain original background)
try:
    # Detect edges using Canny
    edges = cv2.Canny(hand_removed, 1528, 1529)  # Adjust thresholds as needed (low_threshold, high_threshold)

    # Dilate the edges slightly to thicken the lines
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))  # Adjust kernel size for line thickness
    dilated_edges = cv2.dilate(edges, kernel, iterations=2)

    # Create a copy of the original image to overlay green lines
    table_lines_colored = cv2.merge([hand_removed, hand_removed, hand_removed])  # Convert grayscale to BGR

    # Overlay green color on detected table lines
    table_lines_colored[dilated_edges > 0] = [0, 255, 0]  # Green (0, 255, 0) for detected lines

    # Save the resulting image
    step_3_path = os.path.join(output_dir, 'Step 3 - Green Table Lines.png')
    cv2.imwrite(step_3_path, table_lines_colored)

    print(f"Step 3 completed. Green table lines image saved at: {step_3_path}")

except Exception as e:
    print(f"An error occurred in Step 3: {e}")


Step 3 completed. Green table lines image saved at: Berkas Files/images of berkas 500-520 EXAMPLE 2/Preprocessed halaman 1 stuff\Step 3 - Green Table Lines.png


## 3. Memproses teks yang diekstrak

## 4. Save ke dalam file Excel (file format '.csv')